there is another dataset
I think the dataset you worked on is all about the ship
not necessary. I think it is easy to find the coordinates of gas station. So I think this dataset has ship location. We can assume a ship around the gas station is to try to fill the gas. Then track the ship. And based on the ship size, we can calculate how much the oil left in its tank
And of course, we might need some expert knowledge of the MPG of a ship of certain size
the dataset I gave you is one day's data. We might need to rollup multiple days

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

C:\Users\Ulric\Anaconda3\lib\site-packages\IPython\kernel\__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [16]:
pathData = '''D:\\data\\ships\\'''
shipFileName = '''st.txt'''
sailingFileName ='''dy.txt'''

schemaDynamic = ['timestampUNIX',
    'MMSI', # (Maritime Mobile Service Identity)
    'navCondition',
    'steeringRate',
    'speedOverGround',
    'deviceAccuracy',
    'latitude',
    'longitude',
    'gpsCOG', # Course over ground
    'heading',
    'utcVerificationSeconds',
    'packetID',
    'dataSourceID']
schemaStatic = ['timestampUNIX',
    'MMSI', # (Maritime Mobile Service Identity)
    'IMO', # (International Maritime Organization)
    'callsign',
    'vessel',
    'shipType',
    'captain',
    'breadth',
    'pointingDeviceType',
    'eta',
    'currentDraft',
    'destination',
    'packetID',
    'dataSourceID']

In [7]:
#shipData = pd.read_csv(pathData, header=None, engine='c', skiprows=[83988, 137058, 288322])
shipData = pd.read_csv(pathData+shipFileName, header=None, engine='c', quoting=3, error_bad_lines=False)
sailingData = pd.read_csv(pathData+sailingFileName, header=None, engine='c', quoting=3, error_bad_lines=False)

In [17]:
shipData.columns = schemaStatic
sailingData.columns = schemaDynamic

In [10]:
shipData.head()

,timestampUNIX,MMSI,IMO,callsign,vessel,shipType,captain,breadth,pointingDeviceType,eta,currentDraft,destination,packetID,dataSourceID
0,1409501013,219905000,0,XP5174,L68 JANGA BORK,30,18,6,1,1596,30,CH 16 FOR DEST,5,103
1,1409501013,219946000,0,OZYD,HELLE JES,30,21,6,1,1596,0,HUIDB SANDE,5,103
2,1409501013,261182647,0,SR2647,KRYSTYNA,99,62,8,15,568448,2,KALININGRAD *:,5,103
3,1409501013,265623790,0,NaN,EMM XXIII,0,0,0,0,0,0,NaN,24,103
4,1409501013,305477000,9160334,V2EM4,VECHTBORG,70,133,16,1,582656,48,BREMEN BY KIEL CANAL,5,103


In [18]:
sailingData.head()

,timestampUNIX,MMSI,navCondition,steeringRate,speedOverGround,deviceAccuracy,latitude,longitude,gpsCOG,heading,utcVerificationSeconds,packetID,dataSourceID
0,1409501013,205218000,5,0,0,1,-0.071837,53.579707,1482,208,45,3,103
1,1409501013,211596150,0,0,0,1,12.098808,54.181470,3600,511,47,18,103
2,1409501013,231742000,0,0,72,1,-6.759133,62.081663,1401,511,32,18,103
3,1409501013,232000760,0,129,129,0,-2.710517,59.292250,2335,231,47,1,103
4,1409501013,235031438,0,0,0,0,11.849462,57.670968,2716,511,47,18,103


# Join ship data to sailing Data by MMSI

Memory error when joining entire day of sailingData

These data sets need to be in a DB

In [21]:
# Use only a few rows due to memory issues
sampleSize = 100000

In [22]:
sailingShipData = pd.merge(left=sailingData.loc[0:sampleSize, :], right=shipData, how='left', on='MMSI')

In [23]:
sailingShipData.head()

,timestampUNIX_x,MMSI,navCondition,steeringRate,speedOverGround,deviceAccuracy,latitude,longitude,gpsCOG,heading,...,vessel,shipType,captain,breadth,pointingDeviceType,eta,currentDraft,destination,packetID_y,dataSourceID_y
0,1409501013,205218000,5,0,0,1,-0.071837,53.579707,1482,208,...,BNS BELGICA,1,51,10,1,1596,0,NaN,5,101
1,1409501013,205218000,5,0,0,1,-0.071837,53.579707,1482,208,...,BNS BELGICA,1,51,10,1,1596,0,NaN,5,101
2,1409501013,205218000,5,0,0,1,-0.071837,53.579707,1482,208,...,BNS BELGICA,1,51,10,1,1596,0,NaN,5,101
3,1409501013,205218000,5,0,0,1,-0.071837,53.579707,1482,208,...,BNS BELGICA,1,51,10,1,1596,0,NaN,5,101
4,1409501013,205218000,5,0,0,1,-0.071837,53.579707,1482,208,...,BNS BELGICA,1,51,10,1,1596,0,NaN,5,101


In [81]:
#eta is in days?
851436 / 60/ 60 / 24

9.854583333333334

In [82]:
# currentDraft is an 8 bit number?
np.log2(256)

8.0

In [60]:
sum(shipData.iloc[:, 0].isnull())

0

In [45]:
shipData.shape

(3267075, 14)

In [41]:
shipData.loc[83987]

0               1409503140
1                538004244
2                  9218404
3                  V7VZ2  
4     FEDERAL SHIMANTO    
5                       70
6                      190
7                       23
8                        1
9                   592768
10                      60
11                MONTREAL
12                       5
13                     101
Name: 83987, dtype: object

In [42]:
temp[0:4]

['1409501013,219905000,0,XP5174 ,L68 JANGA BORK      ,30,18,6,1,1596,30,CH 16 FOR DEST      ,5,103\n',
 '1409501013,219946000,0,OZYD   ,HELLE JES           ,30,21,6,1,1596,0,HUIDB SANDE         ,5,103\n',
 '1409501013,261182647,0,SR2647,KRYSTYNA ,99,62,8,15,568448,2,KALININGRAD *:,5,103\n',
 '1409501013,265623790,0,,EMM XXIII,0,0,0,0,0,0,,24,103\n']

In [52]:
len(schemaDynamic)

13

In [53]:
temp = shipData.head()
temp.columns = schemaStatic
temp

,UNIX timestamp,Ship MMSI,IMO Ship (International Maritime Organization),Ship callsign,Vessel,The type of ship,captain,Breadth,Pointing device type,ETA,The current draft,destination,Packet ID,Data Source ID
0,1409501013,219905000,0,XP5174,L68 JANGA BORK,30,18,6,1,1596,30,CH 16 FOR DEST,5,103
1,1409501013,219946000,0,OZYD,HELLE JES,30,21,6,1,1596,0,HUIDB SANDE,5,103
2,1409501013,261182647,0,SR2647,KRYSTYNA,99,62,8,15,568448,2,KALININGRAD *:,5,103
3,1409501013,265623790,0,NaN,EMM XXIII,0,0,0,0,0,0,NaN,24,103
4,1409501013,305477000,9160334,V2EM4,VECHTBORG,70,133,16,1,582656,48,BREMEN BY KIEL CANAL,5,103


In [38]:
shipData.head().describe()

,0,1,2,5,6,7,8,9,10,12,13
count,5.000000e+00,5.000000e+00,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5
mean,1.409501e+09,2.544269e+08,1832066.800000,45.800000,46.80000,7.200000,3.600000,230859.200000,16.000000,8.800000,103
std,2.262742e+01,3.590897e+07,4096625.904121,38.771123,53.26068,5.761944,6.387488,314700.804872,21.954498,8.497058,0
min,1.409501e+09,2.199050e+08,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,5.000000,103
25%,1.409501e+09,2.199460e+08,0.000000,30.000000,18.00000,6.000000,1.000000,1596.000000,0.000000,5.000000,103
50%,1.409501e+09,2.611826e+08,0.000000,30.000000,21.00000,6.000000,1.000000,1596.000000,2.000000,5.000000,103
75%,1.409501e+09,2.656238e+08,0.000000,70.000000,62.00000,8.000000,1.000000,568448.000000,30.000000,5.000000,103
max,1.409501e+09,3.054770e+08,9160334.000000,99.000000,133.00000,16.000000,15.000000,582656.000000,48.000000,24.000000,103


In [43]:
temp[-5:]

['1409588028,910060265,0,,                    ,0,0,0,0,0,0,,24,14\n',
 '1409588044,200000054,0,,YU FA 3 HAO A JING  ,30,26,4,0,0,0,,19,14\n',
 '1409588067,200006446,0,,                    ,0,0,0,0,0,0,,24,14\n',
 '1409588086,412467672,0,,YUE DIANYU41178     ,0,0,0,0,0,0,,24,14\n',
 '1409588088,910060265,0,       ,,30,0,0,0,0,0,,24,14\n']

In [37]:
shipData.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
3267070,1409588028,910060265,0,NaN,,0,0,0,0,0,0,NaN,24,14
3267071,1409588044,200000054,0,NaN,YU FA 3 HAO A JING,30,26,4,0,0,0,NaN,19,14
3267072,1409588067,200006446,0,NaN,,0,0,0,0,0,0,NaN,24,14
3267073,1409588086,412467672,0,NaN,YUE DIANYU41178,0,0,0,0,0,0,NaN,24,14
3267074,1409588088,910060265,0,,NaN,30,0,0,0,0,0,NaN,24,14
